In [ ]:
from Fundamentals import Bayesian_Optimization


In [ ]:
def gaussian_model(lam, n_samples):
    
    gaussian_model = pm.Model()
        
    with gaussian_model:

        theta = pm.Normal("theta", mu=lam["mu_1"], sigma=lam["sigma_1"])
            
        Y_obs = pm.Normal("Y_obs", mu=theta, sigma=lam["sigma"], observed=1.)
        
    with gaussian_model:
        
        idata = pm.sample_prior_predictive(samples = n_samples)
        
    return idata


partitions = np.array([[-1000, -5], [-5, -1], [-1, 3], [3, 1000]])

expert_probs = [np.array([0.1, 0.2, 0.6, 0.1])]



BO = Bayesian_Optimization_PO(pymc_sampling_func=gaussian_model,
                              J = 1,
                              alpha=None,
                              target_type="continuous",
                              target_samples=1000)


   

param_names = ["mu_1", "sigma_1", "sigma", "alpha"]
param_bounds = [[-10., 10.], [0.00001, 4.], [0.00001, 4.], [0.1, 20.]]
param_expected_vals = [-1, 3, 0.5, None]
param_weights = [1, 1, 1]
#param_types = ["mean", "std", "std"] ## No need for specifying type for alpha, but it can be done to add a hyperprior for it

   
    
best_params = BO.optimize_hyperparams(param_names=param_names,
                                      param_bounds=param_bounds,
                                      param_expected_vals = param_expected_vals,
                                      param_weights = param_weights,
                                      partitions=partitions,
                                      expert_probs=expert_probs,
                                      n_trials=75)  
